In [3]:
import numpy as np
import pandas as pd
import glob as glob 
import os
import pathlib
import matplotlib.pyplot as plt

import sys
sys.path.append('./radar_mesure')
from radar import RadarMesure

import re
from collections import defaultdict

# Fichiers bruts
file_list = glob.glob('../data/data_excav/F4/*')
data = file_list

# Dictionnaires séparés par fréquence
classic_dict_13 = {}
classic_dict_17 = {}
excav_dict_13  = {}
excav_dict_17  = {}

# Remplissage des dictionnaires
for file in file_list:
    match = re.search(r'v_(\d+)deg', file)
    if not match:
        continue

    angle = int(match.group(1))

    # On crée une instance juste pour lire la fréquence
    rm = RadarMesure(file)
    freq = rm.frequence

    # Affectation
    if "excav" in os.path.split(file)[1]:
        if freq == 13:
            excav_dict_13[angle] = file
        elif freq == 17:
            excav_dict_17[angle] = file
    else:
        if freq == 13:
            classic_dict_13[angle] = file
        elif freq == 17:
            classic_dict_17[angle] = file

# Angles communs pour chaque fréquence
common_angles_13 = sorted(set(classic_dict_13.keys()) & set(excav_dict_13.keys()))
common_angles_17 = sorted(set(classic_dict_17.keys()) & set(excav_dict_17.keys()))

# Création des listes pour chaque fréquence
data_classic_13 = [RadarMesure(classic_dict_13[a]) for a in common_angles_13]
data_excav_13   = [RadarMesure(excav_dict_13[a])  for a in common_angles_13]

data_classic_17 = [RadarMesure(classic_dict_17[a]) for a in common_angles_17]
data_excav_17   = [RadarMesure(excav_dict_17[a])  for a in common_angles_17]

def plot_by_frequency(data_classic_f, data_excav_f, title_prefix=""):
    common_angles_f = [fc.angle_local for fc in data_classic_f]  
    pente = data_classic_f[0].pente

    fig = plt.figure(figsize=(15, 10))
    gs = fig.add_gridspec(2, len(common_angles_f), height_ratios=[3, 1], hspace=0.5)

    sigma_classic = []
    sigma_excav = []

    for i, (file_c, file_e, angle) in enumerate(
            zip(data_classic_f, data_excav_f, common_angles_f)):
        
        df_c = file_c.df.loc[0.5:5]
        df_e = file_e.df.loc[0.5:5]

        sigma_classic.append(file_c.sigma0)
        sigma_excav.append(file_e.sigma0)

        ax = fig.add_subplot(gs[0, i])
        ax.plot(df_c['copol'], df_c.index, label="Classique", color='blue')
        ax.plot(df_e['copol'], df_e.index, label="Excav", color='orange', linestyle='--')

        all_max = max(df_c['copol'].max(), df_e['copol'].max())
        ax.set_xlim(0, all_max * 1.1)
        ax.set_ylim(5, 0)
        ax.set_title(f"{angle-pente :.1f}°")

        if i == 0:
            ax.set_ylabel("Range (m)")
        else:
            ax.set_yticklabels([])

        ax.set_xlabel("Amplitude du signal (AU)")

    # Bas : sigma vs angle
    ax_bottom = fig.add_subplot(gs[1, :])
    ax_bottom.plot([a - pente for a in common_angles_f], sigma_classic,
                   label='$\\sigma$⁰ avec neige', marker='o', color='blue')
    ax_bottom.plot([a - pente for a in common_angles_f], sigma_excav,
                   label='$\\sigma$⁰ apres excavation', marker='x',
                   linestyle='--', color='orange')

    ax_bottom.set_xlabel("Angle (°)")
    ax_bottom.set_ylabel("$\\sigma$⁰ (dB)")
    ax_bottom.set_title(f"{title_prefix} — Évolution de $\\sigma$⁰ selon l’angle incident")
    ax_bottom.grid(True)
    ax_bottom.legend()

    plt.show()


# --- Trace les deux figures indépendantes ---
if data_classic_13:
    plot_by_frequency(data_classic_13, data_excav_13, title_prefix="Fréquence 13 GHz")

if data_classic_17:
    plot_by_frequency(data_classic_17, data_excav_17, title_prefix="Fréquence 17 GHz")
